In [1]:
import tensorflow as tf
import numpy as np

In [3]:
!pip install keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 38.6 MB/s eta 0:00:00


In [4]:
import keras_nlp

Using TensorFlow backend


In [6]:
!wget http://nlp.uoregon.edu/download/embeddings/glove.6B.50d.txt

--2023-11-10 17:10:44--  http://nlp.uoregon.edu/download/embeddings/glove.6B.50d.txt
Resolving nlp.uoregon.edu (nlp.uoregon.edu)... 128.223.8.161, 2607:8400:205e:8:ae1f:6bff:fe93:6364
Connecting to nlp.uoregon.edu (nlp.uoregon.edu)|128.223.8.161|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171350079 (163M) [text/plain]
Saving to: ‘glove.6B.50d.txt’

glove.6B.50d.txt    100%[===================>] 163.41M   105MB/s    in 1.6s    

2023-11-10 17:10:45 (105 MB/s) - ‘glove.6B.50d.txt’ saved [171350079/171350079]



In [8]:
embeddings_index = {}

with open('glove.6B.50d.txt') as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep = ' ')
    embeddings_index[word] = coefs

In [9]:
embeddings_index ['movie']

array([ 0.30824 ,  0.17223 , -0.23339 ,  0.023105,  0.28522 ,  0.23076 ,
       -0.41048 , -1.0035  , -0.2072  ,  1.4327  , -0.80684 ,  0.68954 ,
       -0.43648 ,  1.1069  ,  1.6107  , -0.31966 ,  0.47744 ,  0.79395 ,
       -0.84374 ,  0.064509,  0.90251 ,  0.78609 ,  0.29699 ,  0.76057 ,
        0.433   , -1.5032  , -1.6423  ,  0.30256 ,  0.30771 , -0.87057 ,
        2.4782  , -0.025852,  0.5013  , -0.38593 , -0.15633 ,  0.45522 ,
        0.04901 , -0.42599 , -0.86402 , -1.3076  , -0.29576 ,  1.209   ,
       -0.3127  , -0.72462 , -0.80801 ,  0.082667,  0.26738 , -0.98177 ,
       -0.32147 ,  0.99823 ], dtype=float32)

# Get the movie data

In [10]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000, maxlen=250)

17464789/17464789 [==============================] - 0s 0us/step


In [11]:
x_all = np.append(x_train, x_test)

In [12]:
x_all_padded = tf.keras.utils.pad_sequences(x_all, padding = "post")

In [13]:
word_lookup = tf.keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [14]:
word_lookup['movie']

17

In [15]:
inverted_word_lookup = dict((index +3, word) for (word, index) in word_lookup.items())

inverted_word_lookup[0] = "[PAD]"
inverted_word_lookup[1] = "[START]"
inverted_word_lookup[2] = "[OOV]"
inverted_word_lookup[3] = "[NA]"

In [27]:
def decode(x):
    return " ".join(inverted_word_lookup[i] for i in x)

In [17]:
decode(x_all[1000])

"[START]theviewerleaveswonderingwhyhebotheredtowatchthisoneorwhyforthatmatteranyonebotheredtomakeitthereisnoplotjustrandomscenesofridiculousaction[OOV][OOV]showersceneappealstothemale[OOV]butthat'snotmuchreasontomakeamovie"

## Build embedding model

In [18]:
embedding_matrix = np.zeros((10004, 50))

hits = 0
misses = 0

for i, word in inverted_word_lookup.items():
    if (i >= 10004):
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        print(word)

print("%d hits, %d misses" % (hits, misses))

else's
miyazaki's
victoria's
paul's
chan's
show's
wife's
character's
hadn't
isn't
haven't
wouldn't
its'
she'd
she's
paperhouse
they'll
it's
it'd
daughter's
ted's
ben's
america's
men's
he'll
john's
audience's
30's
mom's
hero's
hasn't
should've
imho
keaton's
they'd
zelah
you'll
smith's
girls'
craven's
feinstone
moore's
eastwood's
kids'
tv's
town's
anyone's
writer's
1960's
kubrick's
husband's
allen's
80's
stewart's
t'aime
boy's
man'
scott's
it´s
bakshi's


person's
you've
verhoeven's
spielberg's
it'll
carpenter's
life's
sister's
family's
who've
director's
where's
city's
author's
man's
friend's
we'd
would've
day's
freddy's
woman's
1930's
can't
ain't
actors'
90's
ossessione
ford's
couldn't
1990's
won't
that'll
other's
aren't
doctor's
everybody's
jackson's
we're
hollywood's
kelly's
david's
murphy's
dvd's
shakespeare's
characters'
mother's
he's
he'd
hitler's
everyone's
don't
could've
child's
miike's
simon's
children's
let's
didn't
you're
bug's
40's
someone's
today's
gypo
lynch's
1950's
palm

### Create a decoder-only transformer

In [19]:
# WORD + POSITION EMBEDDING
input_layer = tf.keras.layers.Input(shape=(None,))
word_embedding = tf.keras.layers.Embedding(10004, 50, embeddings_initializer = tf.keras.initializers.constant(embedding_matrix), trainable=False, mask_zero=True)(input_layer)
position_embedding = keras_nlp.layers.SinePositionEncoding()(word_embedding)
word_and_position_embedding = word_embedding + position_embedding # taking output of these two layers and element wise adds them
# ATTENTION
attention = tf.keras.layers.MultiHeadAttention(10, 50)(word_and_position_embedding, word_and_position_embedding, use_causal_mask=True) # one for query, the other for key (keys and value the same so no need third), causal mask: since we give the model the entire sentence at a time, we want it to not have access to words in the future at a given position in a sentence. So we use this mask to mask out words ahead so the self attention cannot look ahead
residual = tf.keras.layers.Add()([word_and_position_embedding, attention])
normalize = tf.keras.layers.LayerNormalization()(residual)
# DENSE
dense = tf.keras.layers.Dense(50, activation="relu")(normalize)
residual_dense = tf.keras.layers.Add()([normalize, dense])
normalize_dense = tf.keras.layers.LayerNormalization()(residual_dense)

# OUTPUT
linear = tf.keras.layers.Dense(50, activation=None)(normalize_dense)
output_layer = tf.keras.layers.Dense(10004, activation="softmax")(linear)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 50)             500200    ['input_1[0][0]']             
                                                                                                  
 sine_position_encoding (Si  (None, None, 50)             0         ['embedding[0][0]']           
 nePositionEncoding)                                                                              
                                                                                                  
 tf.__operators__.add (TFOp  (None, None, 50)             0         ['embedding[0][0]',       

In [20]:
y=np.roll(x_all_padded, -1, 1)

In [21]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy"
)

In [22]:
model.fit(x=x_all_padded, y=y, epochs=10)

Epoch 1/10
1085/1085 [==============================] - 74s 55ms/step - loss: 3.7208
Epoch 2/10
1085/1085 [==============================] - 64s 59ms/step - loss: 3.3186
Epoch 3/10
1085/1085 [==============================] - 59s 54ms/step - loss: 3.2295
Epoch 4/10
1085/1085 [==============================] - 62s 57ms/step - loss: 3.1808
Epoch 5/10
1085/1085 [==============================] - 60s 55ms/step - loss: 3.1502
Epoch 6/10
1085/1085 [==============================] - 59s 55ms/step - loss: 3.1290
Epoch 7/10
1085/1085 [==============================] - 59s 54ms/step - loss: 3.1131
Epoch 8/10
1085/1085 [==============================] - 61s 56ms/step - loss: 3.1006
Epoch 9/10
1085/1085 [==============================] - 58s 54ms/step - loss: 3.0903
Epoch 10/10
1085/1085 [==============================] - 58s 54ms/step - loss: 3.0819


## Generate predictions

In [33]:
K=500 # number of things we're looking for -> 5 most likely words
result = [1] # 1 = [START]
for i in range(20):
  prediction = model.predict([np.asarray([result])])
  last_token_probability = prediction[0][-1]
  topK = np.argpartition(last_token_probability, -K)[-K:] # dirty sort such that last 5 (index from -1) are most likely, though not necessarily in order
  topK = np.delete(topK, np.where(topK==2))
  topKp = last_token_probability[topK] / np.sum(last_token_probability[topK]) # to get them back to proper probabilities withiin the 5
  result.append(np.random.choice(topK, p=topKp))

1/1 [==============================] - 0s 19ms/step


In [34]:
decode(result)

'[START] i had seen this a terrible piece of watching this which a group in a number of my life so'